In [4]:
from ultralytics import YOLO
import os
import shutil

train_path = "/Users/rukmini/Documents/Project/newdata/reduced_train"
test_path = "/Users/rukmini/Documents/Project/newdata/reduced_test"



def create_yaml_file():
    """Creates a dataset.yaml file for YOLO training."""
    yaml_content = f"""
    train: {train_path}
    val: {test_path}/

    nc: {len(os.listdir(train_path))}  # Number of classes
    names: {os.listdir(train_path)}
    """
    with open("ucf_dataset.yaml", "w") as f:
        f.write(yaml_content)



def generate_dummy_labels(image_folder):
    """Generates dummy label files for images in the specified folder."""
    for class_folder in os.listdir(image_folder):
        class_path = os.path.join(image_folder, class_folder)
        if not os.path.isdir(class_path):
            continue

        class_id = os.listdir(image_folder).index(class_folder)

        for img_file in os.listdir(class_path):
            if img_file.endswith(".png"):
                img_path = os.path.join(class_path, img_file)
                label_file = os.path.splitext(img_path)[0] + ".txt"

                # Generate a dummy label (e.g., full image bounding box)
                dummy_label = f"{class_id} 0.5 0.5 1.0 1.0\n"

                with open(label_file, "w") as f:
                    f.write(dummy_label)



# Step 2: Create the YAML file for the dataset
create_yaml_file()

# Step 3: Generate dummy labels
generate_dummy_labels(train_path)
generate_dummy_labels(test_path)

# Step 4: Load the YOLO model
# You can download a pretrained model or use a custom model
model = YOLO("yolov8n.pt")  # Replace with 'yolov8s.pt', 'yolov8m.pt', etc., for different model sizes

# Step 5: Train the model
model.train(data="ucf_dataset.yaml", epochs=3, imgsz=640, batch=16, project="ucf_project", name="yolo_ucf_train")

# Step 6: Evaluate the model
results = model.val(data="ucf_dataset.yaml")  # Validation on test set

# Step 7: Inference on test videos
def predict_on_test_set(test_folder, model_path):
    """Runs inference on the test folder."""
    results_dir = "test_results"
    if os.path.exists(results_dir):
        shutil.rmtree(results_dir)
    os.makedirs(results_dir)

    for class_folder in os.listdir(test_folder):
        class_path = os.path.join(test_folder, class_folder)
        if not os.path.isdir(class_path):
            continue

        output_class_dir = os.path.join(results_dir, class_folder)
        os.makedirs(output_class_dir, exist_ok=True)

        for img_file in os.listdir(class_path):
            if img_file.endswith(".png"):
                img_path = os.path.join(class_path, img_file)
                result = model.predict(source=img_path, save=True, project=output_class_dir)

# Run inference
predict_on_test_set(test_folder=test_path, model_path="ucf_project/yolo_ucf_train/weights/best.pt")

print("Training, validation, and inference completed.")



Ultralytics 8.3.49 🚀 Python-3.10.9 torch-2.5.1 CPU (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=ucf_dataset.yaml, epochs=3, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=ucf_project, name=yolo_ucf_train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_wi

train: Scanning /Users/rukmini/Documents/Project/newdata/reduced_train/Abuse.cac
val: Scanning /Users/rukmini/Documents/Project/newdata/reduced_test/Abuse.cache.


Plotting labels to ucf_project/yolo_ucf_train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000526, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to ucf_project/yolo_ucf_train2
Starting training for 3 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/3         0G     0.3166      2.002      1.089         14        640: 1
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        557        557      0.431      0.183      0.124      0.122


...

Speed: 3.5ms preprocess, 192.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/NormalVideos/yolo_ucf_train2309

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/NormalVideos/Normal_Videos_935_x264_106760.png: 640x640 1 NormalVideos, 183.5ms
Speed: 5.3ms preprocess, 183.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/NormalVideos/yolo_ucf_train2310

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/NormalVideos/Normal_Videos_935_x264_79160.png: 640x640 1 NormalVideos, 169.5ms
Speed: 2.9ms preprocess, 169.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/NormalVideos/yolo_ucf_train2311

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/NormalVideos/Normal_Videos_911_x264_280.png: 640x640 1 NormalVideos, 212.7ms
Speed: 7.0ms preprocess, 212.7ms inference, 1.7ms postprocess per image at shape (1, 3, 6

Speed: 3.0ms preprocess, 226.4ms inference, 24.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Explosion/yolo_ucf_train22

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Explosion/Explosion002_x264_2980.png: 640x640 1 NormalVideos, 185.0ms
Speed: 3.7ms preprocess, 185.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Explosion/yolo_ucf_train23

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Explosion/Explosion008_x264_410.png: 640x640 1 NormalVideos, 225.4ms
Speed: 9.0ms preprocess, 225.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Explosion/yolo_ucf_train24

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Explosion/Explosion016_x264_50.png: 640x640 1 NormalVideos, 240.8ms
Speed: 4.4ms preprocess, 240.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Expl


image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Explosion/Explosion036_x264_930.png: 640x640 1 NormalVideos, 180.8ms
Speed: 38.7ms preprocess, 180.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Explosion/yolo_ucf_train230

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Explosion/Explosion028_x264_400.png: 640x640 1 NormalVideos, 177.7ms
Speed: 5.1ms preprocess, 177.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Explosion/yolo_ucf_train231

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Arrest/Arrest039_x264_3210.png: 640x640 1 NormalVideos, 174.5ms
Speed: 3.5ms preprocess, 174.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Arrest/yolo_ucf_train2

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Arrest/Arrest039_x264_15750.png: 640x640 1 NormalVideos, 188.3ms
Speed: 3.

Speed: 5.2ms preprocess, 206.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Assault/yolo_ucf_train26

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Assault/Assault010_x264_9310.png: 640x640 1 NormalVideos, 260.8ms
Speed: 2.8ms preprocess, 260.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Assault/yolo_ucf_train27

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Assault/Assault006_x264_4760.png: 640x640 1 NormalVideos, 231.3ms
Speed: 5.9ms preprocess, 231.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Assault/yolo_ucf_train28

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Assault/Assault006_x264_5020.png: 640x640 1 NormalVideos, 201.5ms
Speed: 8.3ms preprocess, 201.5ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Assault/yolo_ucf_tra

Speed: 52.4ms preprocess, 154.9ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Shooting/yolo_ucf_train220

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Shooting/Shooting019_x264_1790.png: 640x640 1 NormalVideos, 150.6ms
Speed: 11.2ms preprocess, 150.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Shooting/yolo_ucf_train221

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Shooting/Shooting047_x264_4560.png: 640x640 1 NormalVideos, 165.5ms
Speed: 3.0ms preprocess, 165.5ms inference, 4.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Shooting/yolo_ucf_train222

image 1/1 /Users/rukmini/Documents/Project/newdata/reduced_test/Shooting/Shooting047_x264_4010.png: 640x640 1 NormalVideos, 243.8ms
Speed: 7.4ms preprocess, 243.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
Results saved to test_results/Shooti

In [2]:
from ultralytics import YOLO
import os
import shutil

train_path = "/Users/rukmini/Documents/Project/newdata/reduced_train"
test_path = "/Users/rukmini/Documents/Project/newdata/reduced_test"

def create_yaml_file():
    """Creates a dataset.yaml file for YOLO training."""
    yaml_content = f"""
    train: {train_path}
    val: {test_path}/

    nc: {len(os.listdir(train_path))}  # Number of classes
    names: {os.listdir(train_path)}
    """
    with open("ucf_dataset.yaml", "w") as f:
        f.write(yaml_content)


def generate_dummy_labels(image_folder):
    """Generates dummy label files for images in the specified folder."""
    for class_folder in os.listdir(image_folder):
        class_path = os.path.join(image_folder, class_folder)
        if not os.path.isdir(class_path):
            continue

        class_id = os.listdir(image_folder).index(class_folder)

        for img_file in os.listdir(class_path):
            if img_file.endswith(".png"):
                img_path = os.path.join(class_path, img_file)
                label_file = os.path.splitext(img_path)[0] + ".txt"

                # Generate a dummy label (e.g., full image bounding box)
                dummy_label = f"{class_id} 0.5 0.5 1.0 1.0\n"

                with open(label_file, "w") as f:
                    f.write(dummy_label)


# Step 1: Create the YAML file for the dataset
create_yaml_file()

# Step 2: Generate dummy labels
generate_dummy_labels(train_path)
generate_dummy_labels(test_path)

# Step 3: Load a lightweight YOLO model
# Use the nano version for better performance on M2
model = YOLO("yolov8n.pt")  # 'yolov8n' for fast performance; replace with 'yolov5n.pt' if needed

# Step 4: Train the model with optimized parameters
model.train(data="ucf_dataset.yaml", epochs=5, imgsz=416, batch=8, project="ucf_project", name="yolo_ucf_train", device="mps")

# Step 5: Evaluate the model
results = model.val(data="ucf_dataset.yaml", device="mps")  # Use MPS backend

# Step 6: Inference on test set
def predict_on_test_set(test_folder, model_path):
    """Runs inference on the test folder."""
    results_dir = "test_results"
    if os.path.exists(results_dir):
        shutil.rmtree(results_dir)
    os.makedirs(results_dir)

    for class_folder in os.listdir(test_folder):
        class_path = os.path.join(test_folder, class_folder)
        if not os.path.isdir(class_path):
            continue

        output_class_dir = os.path.join(results_dir, class_folder)
        os.makedirs(output_class_dir, exist_ok=True)

        for img_file in os.listdir(class_path):
            if img_file.endswith(".png"):
                img_path = os.path.join(class_path, img_file)
                result = model.predict(source=img_path, save=True, project=output_class_dir, device="mps")

# Step 7: Run inference
predict_on_test_set(test_folder=test_path, model_path="ucf_project/yolo_ucf_train/weights/best.pt")

print("Training, validation, and inference completed.")


Ultralytics 8.3.49 🚀 Python-3.10.9 torch-2.5.1 MPS (Apple M2)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=ucf_dataset.yaml, epochs=5, time=None, patience=100, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=mps, workers=8, project=ucf_project, name=yolo_ucf_train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

train: Scanning /Users/rukmini/Documents/Project/newdata/reduced_train/Abuse.cac
val: Scanning /Users/rukmini/Documents/Project/newdata/reduced_test/Abuse.cache.


Plotting labels to ucf_project/yolo_ucf_train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 416 train, 416 val
Using 0 dataloader workers
Logging results to ucf_project/yolo_ucf_train2
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/5      3.21G     0.8105       3.67      1.372         27        416:  


KeyboardInterrupt: 